Take September and as City Milano

In [1]:
import pandas as pd
import numpy as np
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import lars_path
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.metrics import r2_score
import scipy.stats as stats
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [2]:
airbnbOCT= pd.read_csv("/Users/jacopobinati/Desktop/Prediction ML/assignment2/listings_Oct.csv")
airbnbOCT.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,9630344,https://www.airbnb.com/rooms/9630344,20231001204715,2023-10-02,city scrape,Rental unit in Brooklyn · ★4.80 · 1 bedroom ·...,Enjoy your own private bedroom in our 2 bedroo...,Easy access to subway. Trendy and yet quiet.,https://a0.muscache.com/pictures/496cf3f1-7d0d...,47783628,...,5.00,4.80,5.00,NaN,f,1,0,1,0,0.06
1,3533741,https://www.airbnb.com/rooms/3533741,20231001204715,2023-10-02,previous scrape,Rental unit in New York · 1 bedroom · 1 bed · ...,Come stay in the heart of historic Hells Kitch...,NaN,https://a0.muscache.com/pictures/45157664/862a...,17791294,...,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN
2,9731039,https://www.airbnb.com/rooms/9731039,20231001204715,2023-10-02,previous scrape,Rental unit in Queens · Studio · 1 bed · 1 bath,Studio Apartment in the heart of Sunnyside: of...,NaN,https://a0.muscache.com/pictures/81a96b92-d357...,50213378,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
3,21736164,https://www.airbnb.com/rooms/21736164,20231001204715,2023-10-02,previous scrape,Rental unit in Brooklyn · 1 bedroom · 1 bed · ...,"Light-filled, high-ceilinged 1BR brownstone ap...","The neighborhood is vibrant, multicultural, an...",https://a0.muscache.com/pictures/4e3ee5e2-baa1...,4298654,...,5.00,5.00,5.00,NaN,f,1,1,0,0,0.03
4,22280002,https://www.airbnb.com/rooms/22280002,20231001204715,2023-10-01,city scrape,Rental unit in Brooklyn · ★4.98 · 1 bedroom · ...,This is a comfortable super spacious sunny 1 b...,Welcome to the enchanting neighborhood of Stuy...,https://a0.muscache.com/pictures/b187bce7-ded7...,67373899,...,4.91,4.94,4.92,NaN,f,1,1,0,0,1.26


In [3]:
column_info = pd.DataFrame({
    "ID": range(len(airbnbOCT.columns)),
    "Variable": airbnbOCT.columns
})
print(column_info)

    ID                                      Variable
0    0                                            id
1    1                                   listing_url
2    2                                     scrape_id
3    3                                  last_scraped
4    4                                        source
..  ..                                           ...
70  70                calculated_host_listings_count
71  71   calculated_host_listings_count_entire_homes
72  72  calculated_host_listings_count_private_rooms
73  73   calculated_host_listings_count_shared_rooms
74  74                             reviews_per_month

[75 rows x 2 columns]


### Cleaning Dataset
#### After indexing the columns, i dropped some of them (can be checked under). Moreover, I decided to keep a threshold of 40% for varibales with NAs. If we are too narrow, we will reduce the number of variable drastically.

In [4]:
columns_to_drop = [
    0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
    11, 12, 13, 15, 20,
    21,
    44, 45, 46, 47,
    56, 60,
    61, 69
]
airbnb_dropped = airbnbOCT.drop(columns=airbnbOCT.columns[columns_to_drop])
threshold = 0.4
airbnb_nmiss = airbnb_dropped.loc[:, airbnb_dropped.isnull().mean() < threshold]
print(airbnb_nmiss.describe())
airbnb2 = airbnb_nmiss.copy()
airbnb2['price'] = airbnb2['price'].replace('[\$,]', '', regex=True).astype(float)

airbnb2['price'] = pd.to_numeric(airbnb2['price'], errors='coerce')
airbnb2 = airbnb2[airbnb2['price'] <= 10000]
airbnb2['ln_price'] = np.log(airbnb2['price'])

       host_listings_count  host_total_listings_count      latitude   
count         38787.000000               38787.000000  38792.000000  \
mean            141.639286                 228.431021     40.729379   
std             625.516773                 977.805887      0.056725   
min               1.000000                   1.000000     40.500314   
25%               1.000000                   1.000000     40.688790   
50%               2.000000                   3.000000     40.726290   
75%               7.000000                  10.000000     40.762672   
max            4559.000000                8820.000000     40.911380   

          longitude  accommodates          beds  minimum_nights   
count  38792.000000  38792.000000  38129.000000    38792.000000  \
mean     -73.945829      2.907249      1.650214       30.639977   
std        0.055115      2.101448      1.169714       26.633260   
min      -74.251907      1.000000      1.000000        1.000000   
25%      -73.982820      

/Users/jacopobinati/opt/miniconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [5]:
for column in airbnb2.columns:
    value_type = type(airbnb2.loc[0, column])
    print(f"Column '{column}' has values of type: {value_type}")

Column 'host_response_rate' has values of type: <class 'float'>
Column 'host_acceptance_rate' has values of type: <class 'float'>
Column 'host_is_superhost' has values of type: <class 'str'>
Column 'host_thumbnail_url' has values of type: <class 'str'>
Column 'host_listings_count' has values of type: <class 'numpy.float64'>
Column 'host_total_listings_count' has values of type: <class 'numpy.float64'>
Column 'host_verifications' has values of type: <class 'str'>
Column 'host_has_profile_pic' has values of type: <class 'str'>
Column 'host_identity_verified' has values of type: <class 'str'>
Column 'neighbourhood_cleansed' has values of type: <class 'str'>
Column 'neighbourhood_group_cleansed' has values of type: <class 'str'>
Column 'latitude' has values of type: <class 'numpy.float64'>
Column 'longitude' has values of type: <class 'numpy.float64'>
Column 'property_type' has values of type: <class 'str'>
Column 'room_type' has values of type: <class 'str'>
Column 'accommodates' has valu

In [6]:
airbnb2.drop(columns=['room_type'], inplace=True)
airbnb2.drop(columns=['host_thumbnail_url'], inplace=True)
print(airbnb2['review_scores_value'].value_counts)

<bound method IndexOpsMixin.value_counts of 0        5.00
1         NaN
2         NaN
3        5.00
4        4.92
         ... 
38787    4.61
38788    5.00
38789    4.86
38790    4.40
38791    4.67
Name: review_scores_value, Length: 38783, dtype: float64>


In [7]:
airbnb2['property_type'] = airbnb2['property_type'].apply(lambda x: 'Private room' if x.startswith('Private room') else x)
airbnb2['property_type'] = airbnb2['property_type'].apply(lambda x: 'Private room' if x.startswith('Room') else x)
airbnb2['property_type'] = airbnb2['property_type'].apply(lambda x: 'House' if x.startswith('Entire') else x)
airbnb2['property_type'] = airbnb2['property_type'].apply(lambda x: 'Shared Room' if x.startswith('Shared') else x)
property_type_counts = airbnb2['property_type'].value_counts().reset_index()
property_type_counts.columns = ['Property_Type', 'Count']
column_info2 = pd.DataFrame({
    "ID": range(len(airbnbOCT.property_type)),
    "Variable": airbnbOCT.property_type
})
column_info2

allowed_types = ['Private room', 'House', 'Shared Room']
airbnb2 = airbnb2[airbnb2['property_type'].isin(allowed_types)]
property_type_counts

,Property_Type,Count
0,House,21218
1,Private room,17023
2,Shared Room,492
3,Camper/RV,12
4,Tiny home,8
5,Casa particular,6
6,Floor,5
7,Houseboat,5
8,Boat,5
9,Tower,3


In [8]:
property_dummies = pd.get_dummies(airbnb2['property_type'], prefix='property', drop_first=False)
airbnb2 = pd.concat([airbnb2, property_dummies], axis=1)
airbnb2

,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,...,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,ln_price,property_House,property_Private room,property_Shared Room
0,NaN,NaN,f,1.0,2.0,"['email', 'phone']",t,f,Bushwick,Brooklyn,...,5.00,1,0,1,0,0.06,4.174387,False,True,False
1,NaN,NaN,f,1.0,1.0,"['email', 'phone']",t,t,Hell's Kitchen,Manhattan,...,NaN,1,0,1,0,NaN,4.700480,False,True,False
2,NaN,NaN,f,1.0,1.0,"['email', 'phone']",t,f,Sunnyside,Queens,...,NaN,1,1,0,0,NaN,4.595120,True,False,False
3,NaN,NaN,f,1.0,2.0,"['email', 'phone', 'work_email']",t,t,Bedford-Stuyvesant,Brooklyn,...,5.00,1,1,0,0,0.03,4.248495,True,False,False
4,NaN,100%,t,1.0,1.0,"['email', 'phone']",t,t,Bedford-Stuyvesant,Brooklyn,...,4.92,1,1,0,0,1.26,5.135798,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38787,100%,94%,t,1.0,6.0,"['email', 'phone']",t,t,Bedford-Stuyvesant,Brooklyn,...,4.61,1,0,1,0,0.34,4.553877,False,True,False
38788,NaN,NaN,f,2.0,3.0,"['email', 'phone']",t,t,Upper West Side,Manhattan,...,5.00,2,0,2,0,0.01,4.317488,False,True,False
38789,NaN,NaN,f,2.0,3.0,"['email', 'phone']",t,t,Upper West Side,Manhattan,...,4.86,2,0,2,0,0.07,4.442651,False,True,False
38790,90%,100%,t,2.0,6.0,"['email', 'phone']",t,t,Long Island City,Queens,...,4.40,2,2,0,0,0.22,4.976734,True,False,False


### Dealing with Amenities

In [9]:
print(airbnb2['amenities'].head(1))

amenities = airbnb2['amenities'].str.split(',')
amenities_table = pd.Series([item for sublist in amenities for item in sublist]).value_counts()
sorted_table = amenities_table.sort_values(ascending=False)
print(sorted_table.head())

0    ["Body soap", "Dryer", "Refrigerator", "Wifi",...
Name: amenities, dtype: object
 "Wifi"                     35134
 "Kitchen"                  32133
 "Essentials"               31136
 "Carbon monoxide alarm"    29789
 "Hangers"                  27369
Name: count, dtype: int64


### Transform amenities into a binary indicator matrix

In [10]:
amenities_table = airbnb2['amenities'].str.get_dummies(sep=',')
print(amenities_table.head())
amenities_table.describe()

     Bodywash body soap"    Head and Shoulders Dandruff Shampoo shampoo"   
0                      0                                               0  \
1                      0                                               0   
2                      0                                               0   
3                      0                                               0   
4                      0                                               0   

     body soap"   " Bertazzoni stainless steel oven"   
0             0                                    0  \
1             0                                    0   
2             0                                    0   
3             0                                    0   
4             0                                    0   

    " Bose Speaker Bluetooth sound system"   
0                                        0  \
1                                        0   
2                                        0   
3                     

,"Bodywash body soap""","Head and Shoulders Dandruff Shampoo shampoo""","body soap""",""" Bertazzoni stainless steel oven""",""" Bose Speaker Bluetooth sound system""",""" Bowers & Wilkins sound system""",""" Eco Botanics Travel Size conditioner""",""" Eco Botanics Travel Size shampoo""",""" Eco Botanics Travel Size body soap""",""" Eco Botanics Travel Size conditioner""",...,"[""Wifi""","[""Wifi""]","[""Wilpool refrigerator""","[""Window AC unit""","[""Window AC unit""]","[""Window guards""","[""Wine glasses""","[""Xx shampoo""",[],"dove body soap"""
count,38733.000000,38733.000000,38733.000000,38733.000000,38733.000000,38733.000000,38733.000000,38733.000000,38733.000000,38733.000000,...,38733.000000,38733.000000,38733.000000,38733.000000,38733.000000,38733.000000,38733.000000,38733.000000,38733.000000,38733.000000
mean,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000103,0.000077,...,0.003485,0.000129,0.000026,0.000052,0.000026,0.000052,0.000026,0.000077,0.002840,0.000026
std,0.005081,0.005081,0.005081,0.005081,0.005081,0.005081,0.005081,0.005081,0.010162,0.008801,...,0.058935,0.011361,0.005081,0.007186,0.005081,0.007186,0.005081,0.008801,0.053216,0.005081
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Top 10 Amenities in New York City

In [11]:
amenities_nyc = airbnb2['amenities'].str.split(',')
amenities_table_nyc = pd.Series([item for sublist in amenities_nyc for item in sublist]).value_counts()

# Top 10 amenities in NYC
top_10_nyc_amenities = amenities_table_nyc.head(10)
print(top_10_nyc_amenities)

 "Wifi"                     35134
 "Kitchen"                  32133
 "Essentials"               31136
 "Carbon monoxide alarm"    29789
 "Hangers"                  27369
 "Air conditioning"         25828
 "Heating"                  25086
 "Hot water"                24597
 "Smoke alarm"              21631
 "Dishes and silverware"    21169
Name: count, dtype: int64


In [12]:
amenities = [col for col in airbnb2 if col.startswith("d_")]

In [13]:
variable_names = airbnb2.columns.tolist()
variable_table = pd.DataFrame(variable_names, columns=["Variable Names"])
pd.set_option('display.max_rows', None)
print(variable_table)

                                  Variable Names
0                             host_response_rate
1                           host_acceptance_rate
2                              host_is_superhost
3                            host_listings_count
4                      host_total_listings_count
5                             host_verifications
6                           host_has_profile_pic
7                         host_identity_verified
8                         neighbourhood_cleansed
9                   neighbourhood_group_cleansed
10                                      latitude
11                                     longitude
12                                 property_type
13                                  accommodates
14                                bathrooms_text
15                                          beds
16                                     amenities
17                                         price
18                                minimum_nights
19                  

In [15]:
airbnb2.to_csv('/Users/jacopobinati/Desktop/Prediction ML/assignment2/listings_OCT_FINAL.csv', index=False)